In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import regex as re
import numpy as np
import requests
import string
import time 

In [19]:
domain = 'http://amazon.com'
URL = 'https://www.amazon.com/s?k=adidas&i=fashion&bbn=1040658&rh=n%3A7141123011%2Cn%3A7147441011%2Cn%3A1040658%2Cn%3A1045830%2Cp_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&dc&qid=1606922258&rnid=1040658&ref=sr_nr_n_2'
driver = webdriver.Chrome(executable_path ='/Applications/chromedriver')
driver.get(URL)
subhtml = driver.page_source
soup = BeautifulSoup(subhtml, "html.parser")
df = pd.DataFrame()
mylist=[]

while True:
    
    for i in soup.find_all(class_='a-section a-spacing-medium'):
        
        #go inside product page
     
        nextlink = domain + i.find(class_='a-link-normal a-text-normal')['href']
        print(nextlink)
        driver.get(nextlink)
        time.sleep(5)
        temp = driver.page_source
        soup = BeautifulSoup(temp, "html.parser")
        
        #0. extract image link
        try:
            #images = soup.findAll('img')
            images = soup.findAll('div',{'class':'imgTagWrapper'})[0].findAll('img')[0].get('src')
            #for image in images:
            print(images['src'])
            mylist.append(images['src'])
            #image = soup.find(id='imgTagWrapperId').data-a-dynamic-image
            #print(image)            
        except:
            product_image = np.nan
            print('NO product_images')

        #1. extract brand name
        try:
            brand = soup.find_all(id='bylineInfo')[0].text
            if 'Visit' in brand:
                brand = re.match('(?:.{0,10})(.+)(?:\s.+)', brand).group(1)
                print(brand)
            elif 'Brand:' in brand:
                brand = re.match('^Brand\:\s(.+)', brand).group()
            else:
                print('NO brand name')
        except:
            brand = np.nan
            print('NO Brand')
            
        
        #2. extract product name        
        try:   
            product_name = soup.find(id='productTitle').text.replace('\n','').replace(brand+' ','')
            print(product_name)
        except:
            product_name = np.nan
            print('NO product_name')
            
        #3. extract category
        try:
            cate = soup.find(id='wayfinding-breadcrumbs_feature_div').find_all(class_='a-list-item')[6].text.replace('\n','')
            print('Categorie: '+cate)
        except:
            cate = np.nan
            print('NO CATE') 

            
        #4. extract original price /rmb covert to hkd!
        try:
            original_price = soup.find(class_='priceBlockStrikePriceString a-text-strike').text
            print('original price(USD): '+original_price)
            
        except:
            
            try:
                original_price = soup.find(class_='a-size-medium a-color-price priceBlockBuyingPriceString').text
                print('original price(USD): '+original_price)
            except:
                original_price = np.nan
                print('NO original_price')
                
               
                
        #5. extract discounted price /rmb covert to hkd!
        try:
            discounted_price = soup.find(class_='a-size-medium a-color-price priceBlockBuyingPriceString').text
            print('discounted_price(USD): '+discounted_price)
            
        except:
            
            try:
                discounted_price = original_price
                print('discounted_price(USD): '+discounted_price)
            except:
                discounted_price = np.nan
                print('NO discounted_price')
                
        #6. extract promotiona message
#        try:
#            promotion_message = soup.find_all(id='promoPriceBlockMessage_feature_div')[1].text.replace('\n','').replace('  Shop items','')
#            print('Promotion Message: '+promotion_message)
#        except:
#            try:
#                promotion_message = soup.find(id='vpcButton').text.replace('\n','')
#                print('Promotion Message: '+promotion_message)
#            except:
#                promotion_message = np.nan
#                print('NO Promo')
 
        #7. extract no of reviews(ratings)
#        try:
#            no_of_reviews = soup.find(id='acrCustomerReviewText').text.replace(',',"")
#            no_of_reviews = re.match('(\d+)', no_of_reviews).group()
#            print('no_of_reviews: '+no_of_reviews)
#        except:
#            no_of_reviews = np.nan
#            print('NO no_of_reviews')


        #8. extract ratings    
        try:
            ratings = soup.find(class_='a-icon-alt').text
            ratings = re.match('(\d\.?\d?)(?:\s.+)', ratings).group(1)
            print('ratings: ' + ratings)
        except:
            ratings = np.nan
            print('NO ratings')
            
        #9. product url
        try:
            product_url = domain + i.find('a')['href']
            # print(product_url)
        except:
            product_url = np.nan
        
        #column_values = pd.Series(mylist)
        #df = df.insert(loc=0, column='Images', value=column_values)
        
        df = df.append({'Images':images,
                        'Brand':brand,
                        'Product Name':product_name,
                        'Category':cate,
                        'Original Price':original_price,
                        'Discounted Price':discounted_price,
                        'Ratings':ratings,
                        'Product URL':product_url}, ignore_index = True)
#                        'No. of Reviews':no_of_reviews,
#                        'Promotion Message':promotion_message
        
        
    #going back to main page after looping    
    temp = driver.get(URL) 
    subhtml = driver.page_source
    soup = BeautifulSoup(subhtml, "html.parser")

    #going to the next page
    temp2 = soup.find(class_='a-last').a['href']
    URL = domain + temp2
    driver.get(URL)
    subhtml = driver.page_source
    soup = BeautifulSoup(subhtml, "html.parser")
    
    
        

http://amazon.com/adidas-Climastorm-3-Stripes-Jacket-A267/dp/B0835R1Z4L/ref=sr_1_1?dchild=1&keywords=adidas&qid=1606914433&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-1
NO product_images
adidas
Mens Climastorm 3-Stripes Jacket (A267)
Categorie:                 Jackets & Coats            
NO original_price
NO discounted_price
NO ratings
http://amazon.com/adidas-Rink-Jacket-Mens-Hockey/dp/B07BHG5VTC/ref=sr_1_2?dchild=1&keywords=adidas&qid=1606914433&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-2
NO product_images
adidas
Rink Jacket - Men's Hockey
Categorie:                 Men            
original price(USD): $44.65 - $80.00
discounted_price(USD): $44.65 - $80.00
ratings: 4.6
http://amazon.com/adidas-Mens-Essentials-Parka-black/dp/B07ZGDLTXN/ref=sr_1_3?dchild=1&keywords=adidas&qid=1606914433&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-3
NO product_image

NO product_images
adidas
Men's Essentials 3-stripes Woven Windbreaker
Categorie:                 Running            
original price(USD): $27.30 - $87.10
discounted_price(USD): $27.30 - $87.10
ratings: 4.5
http://amazon.com/adidas-outdoor-Mens-Climb-Hoodie/dp/B071445QS4/ref=sr_1_21?dchild=1&keywords=adidas&qid=1606914433&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-21
NO product_images
adidas outdoor mens Dz5984
Categorie:                 Jackets & Coats            
original price(USD): $58.48 - $67.94
discounted_price(USD): $58.48 - $67.94
NO ratings
http://amazon.com/adidas-Originals-Essential-Track-Black/dp/B07X45C6WJ/ref=sr_1_22?dchild=1&keywords=adidas&qid=1606914433&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-22
NO product_images
adidas Originals
mens Essential Track Top
Categorie:                 Active            
original price(USD): $52.49 - $58.25
discounted_price(USD): $52.49 - $58.25

NO product_images
adidas
mens 3-stripes Pullover
Categorie:                 Clothing            
original price(USD): $23.07 - $50.00
discounted_price(USD): $23.07 - $50.00
ratings: 2.0
http://amazon.com/adidas-Originals-Franz-Beckenbauer-Tracktop/dp/B072QBVVPY/ref=sr_1_39?dchild=1&keywords=adidas&qid=1606914433&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-39
NO product_images
adidas Originals
mens Originals Franz Beckenbauer Tracktop
Categorie:                 Men            
original price(USD): $44.99 - $99.00
discounted_price(USD): $44.99 - $99.00
ratings: 4.5
http://amazon.com/adidas-Fielders-Convertible-Baseball-Onix-Core/dp/B07FVHDDJR/ref=sr_1_40?dchild=1&keywords=adidas&qid=1606914433&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-40
NO product_images
adidas
Fielder's Choice 2.0 Convertible Jacket - Men's Baseball
Categorie:                  Jackets & Coats             
original price(USD): 

NO product_images
adidas
Men's Adult Fielders Choice 2.0 Hot Jacket 1/4 Zip Sweatshirt Gray12RA
Categorie:                  Jackets & Coats             
original price(USD): $30.00
discounted_price(USD): $30.00
ratings: 5.0
http://amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_fashion_sr_pg2_1?ie=UTF8&adId=A07720122ZFJY78N9I5V3&url=%2Fadidas-Performance-Hybrid-Jacket-Heather%2Fdp%2FB01N94JQVU%2Fref%3Dsr_1_50_sspa%3Fdchild%3D1%26keywords%3Dadidas%26qid%3D1606915004%26refinements%3Dp_89%253Aadidas%257Cadidas%2BOriginals%257Cadidas%2Boutdoor%26rnid%3D1040658%26s%3Dapparel%26sr%3D1-50-spons%26psc%3D1&qualifier=1606915006&id=1796956803288558&widgetName=sp_atf_next
NO product_images
adidas
BR5917 Men's Nuvic Hybrid
Categorie:                  Jackets & Coats             
original price(USD): $79.95
discounted_price(USD): $79.95
ratings: 5.0
http://amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_fashion_sr_pg2_1?ie=UTF8&adId=A09498211LUNNVXP21WZ2&url=%2Fadid

NO product_images
adidas
Adidas BF Knit Track Jacket
Categorie:                 Active            
original price(USD): $82.92
discounted_price(USD): $82.92
NO ratings
http://amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_mtf_fashion_sr_pg2_1?ie=UTF8&adId=A0824793282X0D83HYDTG&url=%2Fadidas-Athletics-Woven-Anorak-Collegiate%2Fdp%2FB07B3SXSFK%2Fref%3Dsr_1_67_sspa%3Fdchild%3D1%26keywords%3Dadidas%26qid%3D1606915004%26refinements%3Dp_89%253Aadidas%257Cadidas%2BOriginals%257Cadidas%2Boutdoor%26rnid%3D1040658%26s%3Dapparel%26sr%3D1-67-spons%26psc%3D1&qualifier=1606915006&id=1796956803288558&widgetName=sp_mtf
NO product_images
adidas
Athletics ID Woven Anorak - Mens Collegiate Navy CV3269
Categorie:                  Jackets & Coats             
original price(USD): $52.99
discounted_price(USD): $52.99
NO ratings
http://amazon.com/adidas-Team-Track-Jacket-Multi-Sport/dp/B07L2HTDPP/ref=sr_1_68?dchild=1&keywords=adidas&qid=1606915004&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadida

...

NO product_images
adidas
Polar Anorak - Men's
Categorie:                 Jackets & Coats            
original price(USD): $99.95 - $100.00
discounted_price(USD): $99.95 - $100.00
NO ratings
http://amazon.com/Adidas-All-Day-Jacket-Navyroyal/dp/B00GJ9F87G/ref=sr_1_389?dchild=1&keywords=adidas&qid=1606917406&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-389
NO product_images
adidas
Adidas All Day Jacket Navyroyal S
Categorie:                 Active            
original price(USD): $49.99
discounted_price(USD): $49.99
NO ratings
http://amazon.com/adidas-Fleece-Zip-Jacket-Feather/dp/B08MQT2BYF/ref=sr_1_390?dchild=1&keywords=adidas&qid=1606917406&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-390
NO product_images
adidas
Fleece Zip Jacket - Men's
Categorie:                 Active            
original price(USD): $109.95 - $110.00
discounted_price(USD): $109.95 - $110.00
NO ratings
http://amazon.com/gp/slre

NO product_images
adidas
Men's Wb Urb Tasche Bag, Black, NS
Categorie:                  Jackets & Coats             
original price(USD): $38.90
discounted_price(USD): $38.90
ratings: 4.7
http://amazon.com/Adidas-WIND-RDY-Running-Jacket-Apsior/dp/B08BJVPSGM/ref=sr_1_402?dchild=1&keywords=adidas&qid=1606917823&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-402
NO product_images
adidas
Adidas WIND.RDY Running Jacket - AW20 - L - Apsior
Categorie:                 Running            
original price(USD): $227.73
discounted_price(USD): $227.73
NO ratings
http://amazon.com/adidas-Terrex-Windweave-Insulated-Hooded/dp/B08B1Q53Q9/ref=sr_1_403?dchild=1&keywords=adidas&qid=1606917823&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-403
NO product_images
adidas
Adidas Terrex Windweave Insulated Hooded Jacket - AW20 - L - Legink/Actgol.
Categorie:                 Running            
original price(USD): $227.73
disc

NO product_images
adidas
Team Issue Lite Bomber Jacket - Men's Multi-Sport XL Black
Categorie:                  Jackets & Coats             
original price(USD): $52.00
discounted_price(USD): $52.00
NO ratings
http://amazon.com/adidas-Essentials-Tricot-Collegiate-Royal-White/dp/B073V6NTXV/ref=sr_1_423?dchild=1&keywords=adidas&qid=1606917823&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-423
NO product_images
adidas
Essentials 3S Tricot Track Jacket Men's All Sports 2XL Collegiate Royal-White
Categorie:                 Running            
original price(USD): $62.98
discounted_price(USD): $62.98
ratings: 4.8
http://amazon.com/adidas-Celebration-Jacket-Collegiate-Large/dp/B07V7H4997/ref=sr_1_424?dchild=1&keywords=adidas&qid=1606917823&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-424
NO product_images
adidas
mens Bm Celebration Jacket Men
Categorie:                 Running            
original price(US

NO product_images
adidas
Adidas Terrex Agravic Vest - AW20 - Medium - White
Categorie:                 Jackets & Coats            
NO original_price
NO discounted_price
NO ratings
http://amazon.com/adidas-Terrex-Agravic-Vest-Large/dp/B0829HSVT4/ref=sr_1_443?dchild=1&keywords=adidas&qid=1606918181&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-443
NO product_images
adidas
Adidas Terrex Agravic Vest - AW20 - Large - White
Categorie:                 Jackets & Coats            
NO original_price
NO discounted_price
NO ratings
http://amazon.com/adidas-Climastorm-Icon-Knit-Jacket/dp/B072N8CZ5B/ref=sr_1_444?dchild=1&keywords=adidas&qid=1606918181&refinements=p_89%3Aadidas%7Cadidas+Originals%7Cadidas+outdoor&rnid=1040658&s=apparel&sr=1-444
NO product_images
adidas
Adidas Climastorm Icon Knit Jacket - XS - Black
Categorie:                 Running            
NO original_price
NO discounted_price
NO ratings
http://amazon.com/gp/slredirect/picassoRedirec

TypeError: 'NoneType' object is not subscriptable

In [20]:
df.head()

,Brand,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
0,adidas,Jackets & Coats,NaN,https://images-na.ssl-images-amazon.com/images...,NaN,Mens Climastorm 3-Stripes Jacket (A267),http://amazon.com/adidas-Climastorm-3-Stripes-...,NaN
1,adidas,Men,$44.65 - $80.00,https://images-na.ssl-images-amazon.com/images...,$44.65 - $80.00,Rink Jacket - Men's Hockey,http://amazon.com/adidas-Rink-Jacket-Mens-Hock...,4.6
2,adidas,Jackets & Coats,$85.67 - $119.95,https://images-na.ssl-images-amazon.com/images...,$85.67 - $119.95,mens Essentials Down Parka,http://amazon.com/adidas-Mens-Essentials-Parka...,NaN
3,adidas,Jackets & Coats,$41.90 - $69.99,https://images-na.ssl-images-amazon.com/images...,$41.90 - $69.99,Men’s Tech Full Zip Fleece Hoodie Performance ...,http://amazon.com/adidas-Fleece-Hoodie-Perform...,4.6
4,adidas,Jackets & Coats,$44.50 - $53.95,https://images-na.ssl-images-amazon.com/images...,$44.50 - $53.95,mens Team 19,http://amazon.com/adidas-Team-19-Hoody-Multi-S...,4.0


In [21]:
# Check Images Column
df['Images'][0]

'https://images-na.ssl-images-amazon.com/images/I/61ZeWuTAwiL._AC_UY879_.jpg'

In [22]:
# Check Images Column
df['Images'][1]

'https://images-na.ssl-images-amazon.com/images/I/51HzS6ucGOL._AC_UX679_.jpg'

In [23]:
df.shape

(506, 8)

In [24]:
df['Brand'].describe()

count        506
unique         4
top       adidas
freq         418
Name: Brand, dtype: object

In [25]:
# Select the Adidas brand only
df=df[df['Brand'].isin(['adidas', 'adidas Originals', 'adidas outdoor'])]

In [26]:
df.shape

(436, 8)

In [27]:
df.groupby(['Brand']).count()

,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
Brand,,,,,,,
adidas,405,361,418,361,418,418,204
adidas Originals,18,17,18,17,18,18,14


In [28]:
df=df[~df['Discounted Price'].isnull()]
df

,Brand,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
1,adidas,Men,$44.65 - $80.00,https://images-na.ssl-images-amazon.com/images...,$44.65 - $80.00,Rink Jacket - Men's Hockey,http://amazon.com/adidas-Rink-Jacket-Mens-Hock...,4.6
2,adidas,Jackets & Coats,$85.67 - $119.95,https://images-na.ssl-images-amazon.com/images...,$85.67 - $119.95,mens Essentials Down Parka,http://amazon.com/adidas-Mens-Essentials-Parka...,NaN
3,adidas,Jackets & Coats,$41.90 - $69.99,https://images-na.ssl-images-amazon.com/images...,$41.90 - $69.99,Men’s Tech Full Zip Fleece Hoodie Performance ...,http://amazon.com/adidas-Fleece-Hoodie-Perform...,4.6
4,adidas,Jackets & Coats,$44.50 - $53.95,https://images-na.ssl-images-amazon.com/images...,$44.50 - $53.95,mens Team 19,http://amazon.com/adidas-Team-19-Hoody-Multi-S...,4.0
5,adidas,Jackets & Coats,$59.99 - $132.93,https://images-na.ssl-images-amazon.com/images...,$59.99 - $132.93,Skateboarding Light Wind Breaker,http://amazon.com/adidas-Light-Windbreaker-Leg...,NaN
...,...,...,...,...,...,...,...,...
497,adidas,Jackets & Coats,$30.21,https://images-na.ssl-images-amazon.com/images...,$30.21,CORE11 RAIN JKT New Navy/White,http://amazon.com/ADIDAS-CORE11-white-Gr%C3%B6...,3.6
500,adidas,Running,$401.93,https://images-na.ssl-images-amazon.com/images...,$401.93,Adidas Terrex Icesky Hooded Jacket - AW20 - L ...,http://amazon.com/adidas-Terrex-Icesky-Hooded-...,NaN
501,adidas,Other Team Sports,$39.99 - $49.99,https://images-na.ssl-images-amazon.com/images...,$39.99 - $49.99,Anthem Training Jacket - Men,http://amazon.com/Adidas-Anthem-Tricot-Jacket-...,NaN
504,adidas,Running,$69.95,https://images-na.ssl-images-amazon.com/images...,$69.95,ADIDAS ORIGINALS CUT LINE TT TRACK TOP JACKET ...,http://amazon.com/ADIDAS-ORIGINALS-JACKET-Z625...,NaN


In [29]:
df=df[~df['Original Price'].isnull()]
df

,Brand,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
1,adidas,Men,$44.65 - $80.00,https://images-na.ssl-images-amazon.com/images...,$44.65 - $80.00,Rink Jacket - Men's Hockey,http://amazon.com/adidas-Rink-Jacket-Mens-Hock...,4.6
2,adidas,Jackets & Coats,$85.67 - $119.95,https://images-na.ssl-images-amazon.com/images...,$85.67 - $119.95,mens Essentials Down Parka,http://amazon.com/adidas-Mens-Essentials-Parka...,NaN
3,adidas,Jackets & Coats,$41.90 - $69.99,https://images-na.ssl-images-amazon.com/images...,$41.90 - $69.99,Men’s Tech Full Zip Fleece Hoodie Performance ...,http://amazon.com/adidas-Fleece-Hoodie-Perform...,4.6
4,adidas,Jackets & Coats,$44.50 - $53.95,https://images-na.ssl-images-amazon.com/images...,$44.50 - $53.95,mens Team 19,http://amazon.com/adidas-Team-19-Hoody-Multi-S...,4.0
5,adidas,Jackets & Coats,$59.99 - $132.93,https://images-na.ssl-images-amazon.com/images...,$59.99 - $132.93,Skateboarding Light Wind Breaker,http://amazon.com/adidas-Light-Windbreaker-Leg...,NaN
...,...,...,...,...,...,...,...,...
497,adidas,Jackets & Coats,$30.21,https://images-na.ssl-images-amazon.com/images...,$30.21,CORE11 RAIN JKT New Navy/White,http://amazon.com/ADIDAS-CORE11-white-Gr%C3%B6...,3.6
500,adidas,Running,$401.93,https://images-na.ssl-images-amazon.com/images...,$401.93,Adidas Terrex Icesky Hooded Jacket - AW20 - L ...,http://amazon.com/adidas-Terrex-Icesky-Hooded-...,NaN
501,adidas,Other Team Sports,$39.99 - $49.99,https://images-na.ssl-images-amazon.com/images...,$39.99 - $49.99,Anthem Training Jacket - Men,http://amazon.com/Adidas-Anthem-Tricot-Jacket-...,NaN
504,adidas,Running,$69.95,https://images-na.ssl-images-amazon.com/images...,$69.95,ADIDAS ORIGINALS CUT LINE TT TRACK TOP JACKET ...,http://amazon.com/ADIDAS-ORIGINALS-JACKET-Z625...,NaN


In [30]:
df['Discounted Price'] = df['Discounted Price'].str.replace('$', '')
df['Discounted Price']

1       44.65 - 80.00
2      85.67 - 119.95
3       41.90 - 69.99
4       44.50 - 53.95
5      59.99 - 132.93
            ...      
497             30.21
500            401.93
501     39.99 - 49.99
504             69.95
505             94.99
Name: Discounted Price, Length: 378, dtype: object

In [31]:
df['Original Price'] = df['Original Price'].str.replace('$', '')
df['Original Price']

1       44.65 - 80.00
2      85.67 - 119.95
3       41.90 - 69.99
4       44.50 - 53.95
5      59.99 - 132.93
            ...      
497             30.21
500            401.93
501     39.99 - 49.99
504             69.95
505             94.99
Name: Original Price, Length: 378, dtype: object

In [32]:
df.dtypes

Brand               object
Category            object
Discounted Price    object
Images              object
Original Price      object
Product Name        object
Product URL         object
Ratings             object
dtype: object

In [33]:
df['Discounted Price'] = df['Discounted Price'].str.extract(r'(.....)')
df['Discounted Price'] 

1      44.65
2      85.67
3      41.90
4      44.50
5      59.99
       ...  
497    30.21
500    401.9
501    39.99
504    69.95
505    94.99
Name: Discounted Price, Length: 378, dtype: object

In [34]:
df['Original Price'] = df['Original Price'].str.extract(r'(.....)')
df['Original Price'] 

1      44.65
2      85.67
3      41.90
4      44.50
5      59.99
       ...  
497    30.21
500    401.9
501    39.99
504    69.95
505    94.99
Name: Original Price, Length: 378, dtype: object

In [35]:
df['Discounted Price'] = df['Discounted Price'].str.replace(',', '')
df['Discounted Price']

1      44.65
2      85.67
3      41.90
4      44.50
5      59.99
       ...  
497    30.21
500    401.9
501    39.99
504    69.95
505    94.99
Name: Discounted Price, Length: 378, dtype: object

In [36]:
df['Original Price'] = df['Original Price'].str.replace(',', '')
df['Original Price']

1      44.65
2      85.67
3      41.90
4      44.50
5      59.99
       ...  
497    30.21
500    401.9
501    39.99
504    69.95
505    94.99
Name: Original Price, Length: 378, dtype: object

In [37]:
df.to_csv('Amazon_adidas_Jackets3.csv', index = True)

In [38]:
df = pd.read_csv('Amazon_adidas_Jackets3.csv', index_col=0)

In [39]:
df

,Brand,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
1,adidas,Men,44.65,https://images-na.ssl-images-amazon.com/images...,44.65,Rink Jacket - Men's Hockey,http://amazon.com/adidas-Rink-Jacket-Mens-Hock...,4.6
2,adidas,Jackets & Coats,85.67,https://images-na.ssl-images-amazon.com/images...,85.67,mens Essentials Down Parka,http://amazon.com/adidas-Mens-Essentials-Parka...,NaN
3,adidas,Jackets & Coats,41.90,https://images-na.ssl-images-amazon.com/images...,41.90,Men’s Tech Full Zip Fleece Hoodie Performance ...,http://amazon.com/adidas-Fleece-Hoodie-Perform...,4.6
4,adidas,Jackets & Coats,44.50,https://images-na.ssl-images-amazon.com/images...,44.50,mens Team 19,http://amazon.com/adidas-Team-19-Hoody-Multi-S...,4.0
5,adidas,Jackets & Coats,59.99,https://images-na.ssl-images-amazon.com/images...,59.99,Skateboarding Light Wind Breaker,http://amazon.com/adidas-Light-Windbreaker-Leg...,NaN
...,...,...,...,...,...,...,...,...
497,adidas,Jackets & Coats,30.21,https://images-na.ssl-images-amazon.com/images...,30.21,CORE11 RAIN JKT New Navy/White,http://amazon.com/ADIDAS-CORE11-white-Gr%C3%B6...,3.6
500,adidas,Running,401.90,https://images-na.ssl-images-amazon.com/images...,401.90,Adidas Terrex Icesky Hooded Jacket - AW20 - L ...,http://amazon.com/adidas-Terrex-Icesky-Hooded-...,NaN
501,adidas,Other Team Sports,39.99,https://images-na.ssl-images-amazon.com/images...,39.99,Anthem Training Jacket - Men,http://amazon.com/Adidas-Anthem-Tricot-Jacket-...,NaN
504,adidas,Running,69.95,https://images-na.ssl-images-amazon.com/images...,69.95,ADIDAS ORIGINALS CUT LINE TT TRACK TOP JACKET ...,http://amazon.com/ADIDAS-ORIGINALS-JACKET-Z625...,NaN


In [40]:
df['Discounted Price'] = df['Discounted Price'].apply(lambda x: float(x))*7.78 
df['Original Price'] = df['Original Price'].apply(lambda x: float(x))*7.78 
df = df.round(2)
df

,Brand,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
1,adidas,Men,347.38,https://images-na.ssl-images-amazon.com/images...,347.38,Rink Jacket - Men's Hockey,http://amazon.com/adidas-Rink-Jacket-Mens-Hock...,4.6
2,adidas,Jackets & Coats,666.51,https://images-na.ssl-images-amazon.com/images...,666.51,mens Essentials Down Parka,http://amazon.com/adidas-Mens-Essentials-Parka...,NaN
3,adidas,Jackets & Coats,325.98,https://images-na.ssl-images-amazon.com/images...,325.98,Men’s Tech Full Zip Fleece Hoodie Performance ...,http://amazon.com/adidas-Fleece-Hoodie-Perform...,4.6
4,adidas,Jackets & Coats,346.21,https://images-na.ssl-images-amazon.com/images...,346.21,mens Team 19,http://amazon.com/adidas-Team-19-Hoody-Multi-S...,4.0
5,adidas,Jackets & Coats,466.72,https://images-na.ssl-images-amazon.com/images...,466.72,Skateboarding Light Wind Breaker,http://amazon.com/adidas-Light-Windbreaker-Leg...,NaN
...,...,...,...,...,...,...,...,...
497,adidas,Jackets & Coats,235.03,https://images-na.ssl-images-amazon.com/images...,235.03,CORE11 RAIN JKT New Navy/White,http://amazon.com/ADIDAS-CORE11-white-Gr%C3%B6...,3.6
500,adidas,Running,3126.78,https://images-na.ssl-images-amazon.com/images...,3126.78,Adidas Terrex Icesky Hooded Jacket - AW20 - L ...,http://amazon.com/adidas-Terrex-Icesky-Hooded-...,NaN
501,adidas,Other Team Sports,311.12,https://images-na.ssl-images-amazon.com/images...,311.12,Anthem Training Jacket - Men,http://amazon.com/Adidas-Anthem-Tricot-Jacket-...,NaN
504,adidas,Running,544.21,https://images-na.ssl-images-amazon.com/images...,544.21,ADIDAS ORIGINALS CUT LINE TT TRACK TOP JACKET ...,http://amazon.com/ADIDAS-ORIGINALS-JACKET-Z625...,NaN


In [41]:
df.to_csv('Amazon_adidas_Jackets3A.csv', index = True)

In [42]:
df = pd.read_csv('Amazon_adidas_Jackets3A.csv', index_col=0)

In [43]:
df=df[~df['Product Name'].str.contains('Mens', regex=False)]
df

,Brand,Category,Discounted Price,Images,Original Price,Product Name,Product URL,Ratings
1,adidas,Men,347.38,https://images-na.ssl-images-amazon.com/images...,347.38,Rink Jacket - Men's Hockey,http://amazon.com/adidas-Rink-Jacket-Mens-Hock...,4.6
2,adidas,Jackets & Coats,666.51,https://images-na.ssl-images-amazon.com/images...,666.51,mens Essentials Down Parka,http://amazon.com/adidas-Mens-Essentials-Parka...,NaN
3,adidas,Jackets & Coats,325.98,https://images-na.ssl-images-amazon.com/images...,325.98,Men’s Tech Full Zip Fleece Hoodie Performance ...,http://amazon.com/adidas-Fleece-Hoodie-Perform...,4.6
4,adidas,Jackets & Coats,346.21,https://images-na.ssl-images-amazon.com/images...,346.21,mens Team 19,http://amazon.com/adidas-Team-19-Hoody-Multi-S...,4.0
5,adidas,Jackets & Coats,466.72,https://images-na.ssl-images-amazon.com/images...,466.72,Skateboarding Light Wind Breaker,http://amazon.com/adidas-Light-Windbreaker-Leg...,NaN
...,...,...,...,...,...,...,...,...
497,adidas,Jackets & Coats,235.03,https://images-na.ssl-images-amazon.com/images...,235.03,CORE11 RAIN JKT New Navy/White,http://amazon.com/ADIDAS-CORE11-white-Gr%C3%B6...,3.6
500,adidas,Running,3126.78,https://images-na.ssl-images-amazon.com/images...,3126.78,Adidas Terrex Icesky Hooded Jacket - AW20 - L ...,http://amazon.com/adidas-Terrex-Icesky-Hooded-...,NaN
501,adidas,Other Team Sports,311.12,https://images-na.ssl-images-amazon.com/images...,311.12,Anthem Training Jacket - Men,http://amazon.com/Adidas-Anthem-Tricot-Jacket-...,NaN
504,adidas,Running,544.21,https://images-na.ssl-images-amazon.com/images...,544.21,ADIDAS ORIGINALS CUT LINE TT TRACK TOP JACKET ...,http://amazon.com/ADIDAS-ORIGINALS-JACKET-Z625...,NaN
